In [1]:
import jieba

In [ ]:
import numpy as np
import pandas as pd
import joblib

In [3]:
neg = pd.read_excel('neg.xls',header=None)


In [4]:
pos = pd.read_excel('pos.xls',header=None)

In [5]:
neg['words'] = neg[0].apply(lambda x: jieba.lcut(x))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3c/4y6515r52bg3xm59_qswvrrh0000gn/T/jieba.cache
Loading model cost 0.419 seconds.
Prefix dict has been built successfully.


In [6]:
pos['words'] = pos[0].apply(lambda x: jieba.lcut(x))

In [7]:
x = np.concatenate((pos['words'],neg['words']))

In [8]:
y = np.concatenate((np.ones(len(pos)),np.zeros(len(neg))))

In [9]:
from gensim.models.word2vec import Word2Vec

In [10]:
w2v = Word2Vec(size=300,min_count=10)

In [11]:
w2v.build_vocab(x)

In [12]:
w2v.train(x,total_examples=w2v.corpus_count,epochs=10)

(8131792, 11948800)

In [13]:
w2v.save(u'w2v_model.model')

In [14]:
def total_vec(words):
    vec = np.zeros(300).reshape((1,300))
    for word in words:
        try:
            vec += w2v.wv[word].reshape((1,300))
        except KeyError:
            continue
    return vec

In [15]:
train_vec = np.concatenate([total_vec(words) for words in x])

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [17]:
model = SVC(kernel='rbf',verbose=True)

In [18]:
model.fit(train_vec,y)
joblib.dump(model,'svm_model.pkl')

[LibSVM]

['svm_model.pkl']

In [19]:
model = joblib.load('svm_model.pkl')

In [20]:
def total_vec(words):
    w2v = Word2Vec.load('w2v_model.model')
    vec = np.zeros(300).reshape((1,300))
    for word in words:
        try:
            vec += w2v.wv[word].reshape((1,300))
        except KeyError:
            continue
    return vec

In [21]:
def svm_predict():
    df = pd.read_csv("test.csv")
    
    model = joblib.load('svm_model.pkl')
    comment_sentiment = []
    sum_counter = 0;
    right = 0;
    wrong = 0;
    for string in df['内容']:
        words = jieba.lcut(str(string))
        words_vec = total_vec(words)
        result = model.predict(words_vec)
        comment_sentiment.append('积极' if int(result[0]) else '消极')
        
        if sum_counter < 1500:
            if int(result[0]) == 1:
                right += 1
            else:
                wrong += 1
        else:
            if int(result[0]) == 0:
                right += 1
            else:
                wrong += 1
        sum_counter += 1
    
    result = right / sum_counter
    print("判断正确：", right)
    print("判断错误：", wrong)
    print("准确率：", result)

In [22]:
svm_predict()

判断正确： 2599
判断错误： 401
准确率： 0.8663333333333333
